In [ ]:
'''from get_s_and_p_list import get_ticker_list_for_target_date

# define target_date
first_quarter = quarters[0]
target_date = first_quarter.strftime("%Y-%m-%d")

data_dict = {ticker: {} for ticker in tickers}

ticker_list = get_ticker_list_for_target_date(target_date)

for ticker in ticker_list:
    try:
        price = get_closing_price_on_target_date(ticker, target_date)
        intrinsic_value = calculate_graham_number(ticker, target_date)
        data_dict[ticker][target_date] = {'Closing Price': price, 'Graham Number': intrinsic_value}
    except Exception as e:
        print(f'Something went wrong for {ticker} in {target_date}: {e}')

for key, value in data_dict.items():
    print(key)
    print(value)'''

In [1]:
# imports
from quarter_list import quarters
from get_s_and_p_list import get_ticker_list_for_target_date

# define target_date
first_quarter = quarters[0]
target_date = first_quarter.strftime("%Y-%m-%d")

ticker_list = get_ticker_list_for_target_date(target_date)

print(target_date)
print(ticker_list)

         date quarter  year
0  1999-12-31      Q4  1999
1  2000-03-31      Q1  2000
2  2000-06-30      Q2  2000
3  2000-09-30      Q3  2000
4  2000-12-31      Q4  2000
..        ...     ...   ...
88 2021-12-31      Q4  2021
89 2022-03-31      Q1  2022
90 2022-06-30      Q2  2022
91 2022-09-30      Q3  2022
92 2022-12-31      Q4  2022

[93 rows x 3 columns]
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.t

In [2]:
from get_market_data import get_closing_price_on_target_date

for ticker in ticker_list:
    price = get_closing_price_on_target_date(ticker, target_date)
    print(price)

None
0.778050203202955
None


KeyError: 'timestamp'

22.11.23: build pipeline with defining quarters not dates:

In [2]:

import pandas as pd


# start and end date
start_date = "1999-12-31"
end_date = "2022-12-31"

# quarterly: freq='Q'
quarters = pd.date_range(start=start_date, end=end_date, freq='Q')

quarters_df = pd.DataFrame({'date': quarters})

# Extract quarter and year information
quarters_df['quarter'] = quarters_df['date'].dt.to_period("Q")
quarters_df['year'] = quarters_df['date'].dt.year
quarters_df['quarter'] = 'Q' + quarters_df['quarter'].astype(str).str.split('Q').str[1]

print(quarters_df)

         date quarter  year
0  1999-12-31      Q4  1999
1  2000-03-31      Q1  2000
2  2000-06-30      Q2  2000
3  2000-09-30      Q3  2000
4  2000-12-31      Q4  2000
..        ...     ...   ...
88 2021-12-31      Q4  2021
89 2022-03-31      Q1  2022
90 2022-06-30      Q2  2022
91 2022-09-30      Q3  2022
92 2022-12-31      Q4  2022

[93 rows x 3 columns]


In [15]:
from credentials import api_key

# function to get market data for specific date:
def get_closing_price_on_target_date(ticker, key, target_date):
    # endpoint for full output size daily adjusted times series
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&outputsize=full&apikey={key}&datatype=csv'
    csv = pd.read_csv(url)
    df = pd.DataFrame(csv)

    for index, row in df.iterrows():
        try:
            if row['timestamp'] == target_date:
                closing_price = row['close']
                return closing_price
        except:
            print(f'something went wrong getting the {closing_price}')

print(get_closing_price_on_target_date('AAPL', api_key, '2000-01-01'))

       timestamp    open     high      low   close  adjusted_close    volume  \
0     2023-11-22  191.49  192.930  190.825  191.31      191.310000  39630011   
1     2023-11-21  191.41  191.520  189.740  190.64      190.640000  38134485   
2     2023-11-20  189.89  191.905  189.880  191.45      191.450000  46538614   
3     2023-11-17  190.25  190.380  188.570  189.69      189.690000  50941404   
4     2023-11-16  189.57  190.960  188.650  189.71      189.710000  54412915   
...          ...     ...      ...      ...     ...             ...       ...   
6050  1999-11-05   84.62   88.370   84.000   88.31        0.668316   3721500   
6051  1999-11-04   82.06   85.370   80.620   83.62        0.632823   3384700   
6052  1999-11-03   81.62   83.250   81.000   81.50        0.616779   2932700   
6053  1999-11-02   78.00   81.690   77.310   80.25        0.607320   3564600   
6054  1999-11-01   80.00   80.690   77.370   77.62        0.587416   2487300   

      dividend_amount  split_coefficien

In [12]:
from get_fmp_data import get_ratios_from_ticker
import math

def calculate_graham_number(ticker, target_quarter, target_year, key):
    # get ratios dataframe from json zip
    ratio_df = get_ratios_from_ticker('sp-500_Income_Ratio.zip', ticker)

    # iterate over ratios dataframe
    for index, row in ratio_df.iterrows():
        quarter = row['period']
        year = str(row['calendarYear'])

        # extract p/e and p/b at target quarter and year
        if quarter == target_quarter and year == target_year:
            target_date = row['date'].strftime("%Y-%m-%d")
            print(target_date)
            price_to_book_ratio = row['priceToBookRatio']
            print(price_to_book_ratio)
            price_earnings_ratio = row['priceEarningsRatio']
            print(price_earnings_ratio)

            # get the closing price for the same date as in the ratios
            closing_price = get_closing_price_on_target_date(ticker, key, target_date)
            print(closing_price)
'''            try:
                # calculate eps and bvps through ratios
                calculated_eps = closing_price / price_earnings_ratio
                calculated_bvps = closing_price / price_to_book_ratio

                # calculate graham number
                graham_number = math.sqrt(22.5 * calculated_eps * calculated_bvps)
                return graham_number
            except Exception as e:
                print(f'There was a problem calculating graham number: {e}')
                return None

    # If no matching quarter and year is found
    print(f"No matching quarter and year found for {target_quarter} {target_year}")
    return None'''

'            try:\n                # calculate eps and bvps through ratios\n                calculated_eps = closing_price / price_earnings_ratio\n                calculated_bvps = closing_price / price_to_book_ratio\n\n                # calculate graham number\n                graham_number = math.sqrt(22.5 * calculated_eps * calculated_bvps)\n                return graham_number\n            except Exception as e:\n                print(f\'There was a problem calculating graham number: {e}\')\n                return None\n\n    # If no matching quarter and year is found\n    print(f"No matching quarter and year found for {target_quarter} {target_year}")\n    return None'

In [8]:
from credentials import api_key

ticker = 'ABT'
print(calculate_graham_number(ticker, 'Q1', '2000', api_key))

31.881068545802016


In [13]:
ticker_list = ['AAPL', 'ABT', 'ADM', 'ADP', 'ADSK']
for ticker in ticker_list:
    print(calculate_graham_number(ticker, 'Q1', '2000', api_key))

2000-01-01
3.75097266122338
22.618570118360655
None
None
2000-03-31
3.106526279639203
8.824300353412982
35.19
None
2000-03-31
1.135759664882304
16.718543433402242
10.31
None
1999-09-30
5.397072241897272
37.785965937072504
None
None
1999-04-30
10.879103319888303
-102.52441520467836
None
None
